In [1]:
import (
    "fmt"
    "time"
)

type person struct {
    ID           int       `json:"id"`
    GivenName    string    `json:"givenName"`
    FamilyName   string    `json:"familyName"`
    StartDate    time.Time `json:"startDate"`
    Rating       int       `json:"rating"`
}

func parseDate(input string) time.Time {
    const dateForm = "2006-01-02"
    date, _ := time.Parse(dateForm, input)
    return date
}

func getPeople() (people []person) {
    people = append(people, person{1, "John", "Koenig", parseDate("1975-10-17"), 6})
    people = append(people, person{2, "Dylan", "Hunt", parseDate("2000-10-02"), 8})
    people = append(people, person{3, "Leela", "Turanga", parseDate("1999-03-28"), 8})
    people = append(people, person{4, "John", "Crichton", parseDate("1999-03-19"), 7})
    people = append(people, person{5, "Dave", "Lister", parseDate("1988-02-15"), 9})
    people = append(people, person{6, "Laura", "Roslin", parseDate("2003-12-08"), 6})
    people = append(people, person{7, "John", "Sheridan", parseDate("1994-01-26"), 6})
    people = append(people, person{8, "Dante", "Montana", parseDate("2000-11-01"), 5})
    people = append(people, person{9, "Isaac", "Gampu", parseDate("1977-09-10"), 4})
    return
}

func getIds() []int {
    p := getPeople()
    ids := make([]int, 0, len(p))
    for _, n := range p {
        ids = append(ids, n.ID)
    }
    return ids
}

func getPerson(id int) (person, error) {
    time.Sleep(1 * time.Second)
    for _, n := range getPeople() {
        if n.ID == id {
            return n, nil
        }
    }
    return person{}, fmt.Errorf("Cannot find Person ID %d", id)
}


In [5]:
start := time.Now()

for _, id := range getIds() {
    p, _ := getPerson(id)
    fmt.Printf("%d: %s %s\n", id, p.GivenName, p.FamilyName)
}

elapsed := time.Since(start)
fmt.Printf("Total time: %s\n", elapsed)

{} // This is here to keep an extra output message

1: John Koenig
2: Dylan Hunt
3: Leela Turanga
4: John Crichton
5: Dave Lister
6: Laura Roslin
7: John Sheridan
8: Dante Montana
9: Isaac Gampu
Total time: 9.0037614s


In [4]:
import (
    "log"
    "sync"
)

start := time.Now()
ids := getIds()
fmt.Println(ids)

var wg sync.WaitGroup
ch := make(chan person, 10)
for _, n := range ids {
    wg.Add(1)
    go func(id int) {
        defer wg.Done()
        p, err := getPerson(id)
        if err != nil {
            log.Printf("%d: %v\n", id, err)
            return
        }
        ch <- p
    }(n)
}

go func() {
    wg.Wait()
    close(ch)
}()

for p := range ch {
    fmt.Printf("%d: %s %s\n", p.ID, p.GivenName, p.FamilyName)
}

elapsed := time.Since(start)
fmt.Printf("Total time: %s\n", elapsed)

{} // This is here to keep an extra output message

[1 2 3 4 5 6 7 8 9]
9: Isaac Gampu
8: Dante Montana
2: Dylan Hunt
5: Dave Lister
7: John Sheridan
6: Laura Roslin
4: John Crichton
1: John Koenig
3: Leela Turanga
Total time: 1.0071425s
